In [5]:
!pip freeze > requirements.txt


In [12]:


import json
from flask import Flask, request, jsonify,Blueprint
from qdrant_client import QdrantClient, models
from qdrant_client.models import VectorParams, Distance
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import operator
from typing import Annotated, Any
import hashlib
from datetime import datetime
from typing_extensions import TypedDict
from typing import List, Dict, Any
from typing import TypedDict, List, Tuple, Optional
from datetime import datetime
from langgraph.graph import END, StateGraph, START
import ollama
import re
import threading

from collections import deque
from queue import Queue
import socket
import logging
import ast
from IPython.display import Image, display
import PIL.Image as PILImage
from io import BytesIO
from time import sleep,time


from dotenv import load_dotenv

from enum import Enum
# Load environment variables (if using .env)
load_dotenv()
# logging.basicConfig(
#     filename="app.log",
#     filemode="w",
#     format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
#     level=logging.INFO
# )
# logger = logging.getLogger(__name__)


True

In [22]:
images=['', '', '', 'https://b.thumbs.redditmedia.com/f-KMVHQGIVyxH7EsC7_AOwYf6VKIrTTfSOjBCXQEjDQ.jpg', 'https://a.thumbs.redditmedia.com/wHfVo7oym_AVZx1CO0iIe_MELGjhGA4yhhmhlVHxT18.jpg', '', 'https://media.licdn.com/dms/image/v2/D4D05AQHeoxFCvwfVOw/feedshare-thumbnail_720_1280/feedshare-thumbnail_720_1280/0/1728303469612?e=1741723200&v=beta&t=0gk_ZGEDS-SiORL6i9cW40bqpCtckyUKBjV4hM6xZkQ', 'https://media.licdn.com/dms/image/v2/D4D22AQHRjMlbZ1xL0w/feedshare-shrink_480/feedshare-shrink_480/0/1724925368536?e=1744243200&v=beta&t=jyA8HZyQ_wAlg4z3x8GebgIUy3ixSHGzYJxWyrtn-fU', 'https://media.licdn.com/dms/image/sync/v2/D4E10AQHOUYIxX-PRWg/image-shrink_800/image-shrink_800/0/1724231310038/ICSIDReviewWPLinkedInCard3png?e=1741723200&v=beta&t=JLwTOaAHdprxwZC7dsj_EsKvXl5Vhzy6-Gi-bTvQxxQ', 'https://media.licdn.com/media/AAYQAQQSAAgAAQAAAAAAAB1e9tHgbkd0TBCrxo30JVEXCA.jpg', 'https://media.licdn.com/dms/image/v2/D4D05AQHYG7SBRaWWyA/videocover-low/videocover-low/0/1724338176965?e=1741723200&v=beta&t=l2FEOQ1xjHc04KaMb6mzxf31QL5T9Mfdea4qOysA3Ak', 'https://media.licdn.com/dms/image/v2/D4D05AQG3EgL2DJevAw/videocover-high/videocover-high/0/1724844029995?e=1741723200&v=beta&t=4hkOjNkv6nS4rBC9uaekA8KfWP0JeJDQXAaAjeia7pQ', 'https://media.licdn.com/dms/image/sync/v2/D4D10AQF1uPA88n7cUw/image-shrink_800/image-shrink_800/0/1721769843972/FTK-81-Mac-Collections-Ads-ABM-Campaign-1200x628-SOCIAL-11gif?e=1741723200&v=beta&t=Hv1Im9OLDotVAkztMKYv1KxyXyI3NEp1H6zBOu1ct0E', 'https://media.licdn.com/dms/image/v2/D4D22AQGNeVVD6Rirqg/feedshare-shrink_480/feedshare-shrink_480/0/1724588813666?e=1744243200&v=beta&t=lIgR-ce_QJ_7i1arbkMgT1kQYwqC1WZjEw1-euug26E', 'https://media.licdn.com/dms/image/v2/D4D1FAQHRAoQmJDGLPA/feedshare-document-cover-images_480/feedshare-document-cover-images_480/0/1724677618702?e=1741723200&v=beta&t=GgJaFK331bVHGrJgT_B49BkH0EWD8u1z_7EdtqlSlBc', 'https://media.licdn.com/dms/image/v2/D4D1FAQG7pBHkeK1r7g/feedshare-document-cover-images_480/feedshare-document-cover-images_480/0/1724854203346?e=1741723200&v=beta&t=xQjWOlPFaH24LNWMrTadKsjFPveyFtA0qMdcVPscXxY', 'https://media.licdn.com/dms/image/sync/v2/D4D10AQF_CG74PVrI7g/image-shrink_800/image-shrink_800/0/1721336398045/AZRDIR_LI-FreeAccount-NA-2_IND_1200x1200_LIA_AZR-Direct_EN_Trial_Standard_SBAN_SINU_Lifestyle_Statementpng?e=1741723200&v=beta&t=PJW2qPNgCQRaOZbIUpjm_-HrKJyP5lMNbFTiSZ6oOYM', 'https://media.licdn.com/dms/image/v2/D4D22AQG_rtmYBaf6Xg/feedshare-shrink_480/feedshare-shrink_480/0/1724940189293?e=1744243200&v=beta&t=WQnikeEDu7B5Rjk8_pAPY3ixA7ykBlwdQYRKAJfn6pQ', 'https://media.licdn.com/dms/image/v2/D4D10AQHgvLUo1EydKA/videocover-high/videocover-high/0/1724851790192?e=1741723200&v=beta&t=0-XojhSZhBIYb_WS4lZ4cYld5ewTiw4s1SlDAAdWDV0', 'https://media.licdn.com/dms/image/v2/D4D1FAQGypc_h8_9QLQ/feedshare-document-cover-images_480/feedshare-document-cover-images_480/0/1724939957781?e=1741723200&v=beta&t=rkRZE0Dyeo5e25YTXE-A9-LnouP1ih2nEF-UPOmRPTk', 'https://scontent.flhe4-2.fna.fbcdn.net/v/t39.30808-6/480243915_957217613065458_4254873056270645372_n.jpg?stp=c0.213.1284.1284a_dst-jpg_s206x206_tt6&_nc_cat=107&ccb=1-7&_nc_sid=e21142&_nc_eui2=AeH_XKokhAY6f6xzV1ArwNkiZLa7ygch6VpktrvKByHpWkE6bQR5bP-Xy_CJiDJBGCs7ZDwCkZJEAcfOax-2elT1&_nc_ohc=i7fk9VWkYwUQ7kNvgE_pXI2&_nc_oc=AdgjySJfmHD7EUMq3ih3fQQpeGsqic_tjbB8ex5-9cQn5ioakCjWbKxMXcVpWTc1W66cZLUlQIQcxM6DRBoR44R4&_nc_zt=23&_nc_ht=scontent.flhe4-2.fna&_nc_gid=AZsnV_X3jhaoELvJX4TeGI_&oh=00_AYHDPtschnXXCsO-9W2oa81CpBTVn70s5lJ8eGtpa4EC5A&oe=67D83F6F', 'https://scontent.flhe4-1.fna.fbcdn.net/v/t39.30808-6/481964022_1044104501082959_6079336484102729211_n.jpg?stp=c120.0.720.720a_cp6_dst-jpg_s206x206_tt6&_nc_cat=108&ccb=1-7&_nc_sid=e21142&_nc_eui2=AeEJ4na0hxWe662hUES9Ds763pL9vaJChiXekv29okKGJUigWX7noZlNB2KGEt2GO0RIO24Uqe7EjVQxAnXkyhrg&_nc_ohc=ZPN7MGgw2AgQ7kNvgHqxL4g&_nc_oc=AdgtGpOlYU-8zpfwCahULif-Eh6APSOHFSunOMlDlx9G0mD4NGSzOrJnoRx9vIBPKnaM6ZMQmSBD2bFdYb8-QaQe&_nc_zt=23&_nc_ht=scontent.flhe4-1.fna&_nc_gid=AZsnV_X3jhaoELvJX4TeGI_&oh=00_AYHRYoiXLIYw7-_gZgGgQIoo9kdmHyKDYiSS0UkOEvG9_g&oe=67D836D5', 'https://scontent.flhe4-2.fna.fbcdn.net/v/t39.30808-6/482253931_2000502973773750_3910079502931543773_n.jpg?stp=dst-jpg_s206x206_tt6&_nc_cat=111&ccb=1-7&_nc_sid=e21142&_nc_eui2=AeFjupFmo2jm7IMken1uC1D5XUYab7TJ8LBdRhpvtMnwsDW1xd4eAlnNJcoLEwQhnMuDZH3JIFfxd_e44gHvzWY5&_nc_ohc=0b4p2HYHILwQ7kNvgGUUJEJ&_nc_oc=AdjBg_YWDbKzaMW4G4j7B-B9BoeJnmwzd6rZAcI0JvAyi0qF9FPFZyW8R9e0rPQ7YxFUqb1fydgSZ4noyu23frvA&_nc_zt=23&_nc_ht=scontent.flhe4-2.fna&_nc_gid=AZsnV_X3jhaoELvJX4TeGI_&oh=00_AYFR8qBmg427G4SB8wbh_dVeexVnso57GXQCdOtkhAsYsg&oe=67D83CE2', 'https://scontent.flhe4-2.fna.fbcdn.net/v/t39.30808-6/482061667_122188320812121935_3363345127891881338_n.jpg?stp=c0.293.1414.1414a_dst-jpg_s206x206_tt6&_nc_cat=1&ccb=1-7&_nc_sid=e21142&_nc_eui2=AeF4caRVBaca75L52d6x7o1tAvYyLatHzPgC9jItq0fM-JB1tThL5T2n6Wnu3-jxcIh98d_GH1aRxi79qpVXyzIb&_nc_ohc=u-gGIO_PzaEQ7kNvgGHfZha&_nc_oc=Adg0H7OQrIKxeQLsjlDzd2PoH-hLLN5_MgKtqRdoO1Gq8ST9hMjC_Z-MwBIVUAHu1p-nI-Y135HVLM5xfC_8JS0_&_nc_zt=23&_nc_ht=scontent.flhe4-2.fna&_nc_gid=AZsnV_X3jhaoELvJX4TeGI_&oh=00_AYHtfgGTQVvamQv2eRptD4U-tgwR9HCKIQqV4TfbcXnuEw&oe=67D83CA0', 'https://scontent.flhe4-2.fna.fbcdn.net/v/t39.30808-1/445209194_862442485918806_1586323370714697053_n.jpg?stp=c33.33.294.294a_dst-jpg_s200x200_tt6&_nc_cat=1&ccb=1-7&_nc_sid=fe756c&_nc_eui2=AeGjwTZPT517DZGeQr020s0q2VIp3tk6bCXZUine2TpsJTdfX9udllbmM-ALZ4A76Te5GPaQpFtuhvI5w0CUv5Jx&_nc_ohc=K-eEZjMxNaIQ7kNvgFkxHaH&_nc_oc=AdhdI-VJffs3HQnxEQkEn4lzI0j9Xv3f8i7kY-4hLZlpc5fMC1e2A4cOQdpPc6xPesqdUME8GMfVgHkTnaIT-EIK&_nc_zt=24&_nc_ht=scontent.flhe4-2.fna&_nc_gid=AZsnV_X3jhaoELvJX4TeGI_&oh=00_AYGj_Mqd1e0eJ6Xm8JHcz99Ht99TTcyoRApc4zlMctoXmg&oe=67D84A2F', 'https://scontent.flhe4-1.fna.fbcdn.net/v/t39.30808-6/483368200_122160512564436890_4086727410595556976_n.jpg?stp=c0.135.1080.1080a_dst-jpg_s206x206_tt6&_nc_cat=106&ccb=1-7&_nc_sid=e5c1b6&_nc_eui2=AeEOOHfnkMjybhb7iC9kIWQ6eis7MXJtyJF6Kzsxcm3IkS0Vlc7CxVEadXZcz9twYFkaOUZ6nOM6cga6fMgyFr5l&_nc_ohc=0sj39kapDR8Q7kNvgEIdk5P&_nc_oc=AdiHdXHUfOfzWZJKOa8zbtUENfmHr7oRD0_QbKTn-pn5qSEzJbLy3mv6P9lzcB2HjorLE4ZFcJn63hvkKvZ-Dz_e&_nc_zt=23&_nc_ht=scontent.flhe4-1.fna&_nc_gid=AZsnV_X3jhaoELvJX4TeGI_&oh=00_AYGEEfviw3vog4CZ9kfUtTBA81bmxSX56gY3A0XFYQv9gg&oe=67D85C68', 'https://scontent-sin2-1.xx.fbcdn.net/v/t39.30808-1/445209194_862442485918806_1586323370714697053_n.jpg?stp=c33.33.294.294a_dst-jpg_s200x200_tt6&_nc_cat=1&ccb=1-7&_nc_sid=fe756c&_nc_ohc=K-eEZjMxNaIQ7kNvgEIJIWn&_nc_zt=24&_nc_ht=scontent-sin2-1.xx&_nc_gid=ALimGUX4EC0XSBb8gOdqtrz&oh=00_AYF4YVrs3h-xD9tbfb3_x9rKkHJE5jE49geBCczv8SnnoA&oe=67D84A2F', 'https://scontent-sin2-3.xx.fbcdn.net/v/t39.30808-6/480243915_957217613065458_4254873056270645372_n.jpg?stp=c0.213.1284.1284a_dst-jpg_s206x206_tt6&_nc_cat=107&ccb=1-7&_nc_sid=e21142&_nc_ohc=i7fk9VWkYwUQ7kNvgFpEtcZ&_nc_zt=23&_nc_ht=scontent-sin2-3.xx&_nc_gid=ALimGUX4EC0XSBb8gOdqtrz&oh=00_AYH-tdT5xOxhNDXsq0XEFSaeHz66EBdeghAKKSf8WwB65A&oe=67D83F6F', 'https://scontent-sin6-3.xx.fbcdn.net/v/t39.30808-6/483368200_122160512564436890_4086727410595556976_n.jpg?stp=c0.135.1080.1080a_dst-jpg_s206x206_tt6&_nc_cat=106&ccb=1-7&_nc_sid=e5c1b6&_nc_ohc=0sj39kapDR8Q7kNvgEM6xzZ&_nc_zt=23&_nc_ht=scontent-sin6-3.xx&_nc_gid=ALimGUX4EC0XSBb8gOdqtrz&oh=00_AYF6pcdELDqHcPQZDuuwHfMl3GXxIDU0q4pAYnfXxqqcqw&oe=67D85C68', 'https://scontent-sin11-2.xx.fbcdn.net/v/t39.30808-6/481964022_1044104501082959_6079336484102729211_n.jpg?stp=c120.0.720.720a_cp6_dst-jpg_s206x206_tt6&_nc_cat=108&ccb=1-7&_nc_sid=e21142&_nc_ohc=ZPN7MGgw2AgQ7kNvgGeqNUP&_nc_zt=23&_nc_ht=scontent-sin11-2.xx&_nc_gid=ALimGUX4EC0XSBb8gOdqtrz&oh=00_AYF-sOkE9WioOYylbbm9cIZm7tQu_Ieh_73WTS7Xkn0AqA&oe=67D836D5', 'https://scontent-sin2-1.xx.fbcdn.net/v/t39.30808-6/482061667_122188320812121935_3363345127891881338_n.jpg?stp=c0.293.1414.1414a_dst-jpg_s206x206_tt6&_nc_cat=1&ccb=1-7&_nc_sid=e21142&_nc_ohc=u-gGIO_PzaEQ7kNvgEqIpE_&_nc_zt=23&_nc_ht=scontent-sin2-1.xx&_nc_gid=ALimGUX4EC0XSBb8gOdqtrz&oh=00_AYGtKQh4rqZIMiAe7bR_AgYofBrdkRlCfkyZUJC120xD6g&oe=67D83CA0', 'https://scontent-sin6-1.xx.fbcdn.net/v/t39.30808-6/482253931_2000502973773750_3910079502931543773_n.jpg?stp=dst-jpg_s206x206_tt6&_nc_cat=111&ccb=1-7&_nc_sid=e21142&_nc_ohc=0b4p2HYHILwQ7kNvgGYR213&_nc_zt=23&_nc_ht=scontent-sin6-1.xx&_nc_gid=ALimGUX4EC0XSBb8gOdqtrz&oh=00_AYHAlCeTr5VRyapKPuacFWUFmq2FKzRW2n3gMIvqpbdE9Q&oe=67D83CE2']

In [23]:
import os
import urllib.request

# List of image URLs


# Create a folder to store images
folder_name = "downloaded_images"
os.makedirs(folder_name, exist_ok=True)  # Create folder if it doesn't exist

# Download and save images
for i, img in enumerate(images):
    filename = os.path.join(folder_name, f"downloaded_image_{i+1}.jpg")  # Store in folder
    print(f"Downloading: {img}")

    try:
        urllib.request.urlretrieve(img, filename)
        print(f"Saved: {filename}")
    except Exception as e:
        print(f"Failed to download {img}: {e}")


Downloading: 
Failed to download : unknown url type: ''
Downloading: 
Failed to download : unknown url type: ''
Downloading: 
Failed to download : unknown url type: ''
Downloading: https://b.thumbs.redditmedia.com/f-KMVHQGIVyxH7EsC7_AOwYf6VKIrTTfSOjBCXQEjDQ.jpg
Saved: downloaded_images\downloaded_image_4.jpg
Downloading: https://a.thumbs.redditmedia.com/wHfVo7oym_AVZx1CO0iIe_MELGjhGA4yhhmhlVHxT18.jpg
Saved: downloaded_images\downloaded_image_5.jpg
Downloading: 
Failed to download : unknown url type: ''
Downloading: https://media.licdn.com/dms/image/v2/D4D05AQHeoxFCvwfVOw/feedshare-thumbnail_720_1280/feedshare-thumbnail_720_1280/0/1728303469612?e=1741723200&v=beta&t=0gk_ZGEDS-SiORL6i9cW40bqpCtckyUKBjV4hM6xZkQ
Failed to download https://media.licdn.com/dms/image/v2/D4D05AQHeoxFCvwfVOw/feedshare-thumbnail_720_1280/feedshare-thumbnail_720_1280/0/1728303469612?e=1741723200&v=beta&t=0gk_ZGEDS-SiORL6i9cW40bqpCtckyUKBjV4hM6xZkQ: HTTP Error 403: Forbidden
Downloading: https://media.licdn.com/dm

In [24]:
import os
import google.generativeai as genai
import PIL.Image
from dotenv import load_dotenv  

# Load environment variables
load_dotenv()

# Ensure API key is set
KEY = os.getenv("GEMINI_API_KEY")
if not KEY:
    raise ValueError("GEMINI_API_KEY is not set. Please check your environment variables.")

# Configure Gemini API
genai.configure(api_key=KEY)

# Folder containing downloaded images
image_folder = "downloaded_images"

# Check if the folder exists
if not os.path.exists(image_folder):
    raise FileNotFoundError(f"Folder '{image_folder}' not found. Please check the image download process.")

# Get list of all images in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith((".jpg", ".png", ".jpeg"))]

if not image_files:
    raise FileNotFoundError("No images found in the folder!")

# Initialize Gemini model
model = genai.GenerativeModel("gemini-2.0-flash")  # Correct model name

# Process each image one by one
for img_file in image_files:
    image_path = os.path.join(image_folder, img_file)

    print(f"Processing image: {image_path}")

    # Load image
    image = PIL.Image.open(image_path)

    # Generate response
    response = model.generate_content(["What is this image?", image])

    # Print response
    print(f"Response for {img_file}: {response.text}")
    print("=" * 80)  # Separator for better readability


Processing image: downloaded_images\downloaded_image_10.jpg
Response for downloaded_image_10.jpg: The image appears to be a celebratory graphic, likely for a birthday or other festive event. It features a large, smiling, sun-like balloon and other white and blue balloons. The background has a light blue tone with what appear to be small dots.
Processing image: downloaded_images\downloaded_image_14.jpg
Response for downloaded_image_14.jpg: The image shows a list of freelance job postings related to web development, specifically for WordPress and Zoho Sites. Each entry includes the job title, a 5-star rating, dates, price, and snippets of client feedback.

Processing image: downloaded_images\downloaded_image_18.jpg
Response for downloaded_image_18.jpg: The image shows a man in a suit receiving an award from a woman on a stage. There are banners with company logos behind them. The award event appears to be related to design.
Processing image: downloaded_images\downloaded_image_21.jpg
Resp

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [4]:
import urllib.request

url = "https://b.thumbs.redditmedia.com/f-KMVHQGIVyxH7EsC7_AOwYf6VKIrTTfSOjBCXQEjDQ.jpg"
filename = "downloaded_image.jpg"

# Download and save the image
urllib.request.urlretrieve(url, filename)

print(f"Image saved as {filename}")


Image saved as downloaded_image.jpg


In [4]:

from pymongo import MongoClient
from bson import ObjectId
from dotenv import load_dotenv
# Load environment variables (if using .env)
load_dotenv()
import os
# Get MongoDB URI from environment variable
uri = os.getenv("MONGO_URI")
# Connect to MongoDB
client = MongoClient(uri)
db = client["postbuddy"]
collection = db["user"]





user_id = ObjectId("67c75778bf5515f0b0ccddc2")

user_doc = collection.find_one(
    {"_id": user_id},
    {"posts": 1, "_id": 0}
)

if user_doc and "posts" in user_doc:
    post_ids = user_doc["posts"]
    print(post_ids)
else:
    print("No posts found or user does not exist.")


# # Remove specific fields from all documents
# collection.update_many({}, {
#     "$unset": {
#         "userId": "",
#         "last_fetched": "",
#         "createdAt": "",
#         "updatedAt": ""
#     }
# })

# print("Fields removed successfully from all documents!")


[ObjectId('67c756e2bf5515f0b0ccddbb'), ObjectId('67c756e2bf5515f0b0ccddbd'), ObjectId('67c756e3bf5515f0b0ccddbf'), ObjectId('67c756e2bf5515f0b0ccddb5'), ObjectId('67c756e2bf5515f0b0ccddb7'), ObjectId('67c756e2bf5515f0b0ccddb9'), ObjectId('67c75885bf5515f0b0ccddf1'), ObjectId('67c75885bf5515f0b0ccddf3'), ObjectId('67c75885bf5515f0b0ccddf5'), ObjectId('67c75885bf5515f0b0ccddf7'), ObjectId('67c75886bf5515f0b0ccddf9'), ObjectId('67c75886bf5515f0b0ccddfb'), ObjectId('67c75887bf5515f0b0ccde0c'), ObjectId('67c75887bf5515f0b0ccde0a'), ObjectId('67c75887bf5515f0b0ccde0e'), ObjectId('67c75887bf5515f0b0ccde13'), ObjectId('67c75893bf5515f0b0ccde31'), ObjectId('67c75893bf5515f0b0ccde33'), ObjectId('67c75893bf5515f0b0ccde39'), ObjectId('67c75893bf5515f0b0ccde3b'), ObjectId('67d27b44cbe1d16869afdcc4'), ObjectId('67d27b44cbe1d16869afdcc6'), ObjectId('67d27b45cbe1d16869afdccf'), ObjectId('67d27b45cbe1d16869afdcd1'), ObjectId('67d27b44cbe1d16869afdcc2'), ObjectId('67d27b44cbe1d16869afdcc2'), ObjectId('6

In [5]:
print(len(post_ids))

39


In [15]:
from pymongo import MongoClient
# Load environment variables (if using .env)
load_dotenv()
# Get MongoDB URI from environment variable
uri = os.getenv("MONGO_URI")
# Connect to MongoDB
client = MongoClient(uri)
db = client["postbuddy"]
collection = db["posts"]

# Remove specific fields from all documents
collection.update_many({}, {
    "$unset": {
        "userId": "",
        "last_fetched": "",
        "createdAt": "",
        "updatedAt": ""
    }
})

print("Fields removed successfully from all documents!")


Fields removed successfully from all documents!


In [25]:
doc = collection.find_one()
print(type(doc["_id"]))  # Check the data type


<class 'bson.objectid.ObjectId'>


In [26]:
# Retrieve only the `_id` field from all documents
ids = collection.find({}, {"_id": 1})  # 1 means include `_id`, exclude others

# Convert to list
id_list = [doc["_id"] for doc in ids]

# Print all IDs
print(id_list)

[ObjectId('67c756e2bf5515f0b0ccddb5'), ObjectId('67c756e2bf5515f0b0ccddb7'), ObjectId('67c756e2bf5515f0b0ccddb9'), ObjectId('67c756e2bf5515f0b0ccddbb'), ObjectId('67c756e2bf5515f0b0ccddbd'), ObjectId('67c756e3bf5515f0b0ccddbf'), ObjectId('67c75885bf5515f0b0ccddf1'), ObjectId('67c75885bf5515f0b0ccddf3'), ObjectId('67c75885bf5515f0b0ccddf5'), ObjectId('67c75885bf5515f0b0ccddf7'), ObjectId('67c75886bf5515f0b0ccddf9'), ObjectId('67c75886bf5515f0b0ccddfb'), ObjectId('67c75887bf5515f0b0ccde0a'), ObjectId('67c75887bf5515f0b0ccde0c'), ObjectId('67c75887bf5515f0b0ccde0e'), ObjectId('67c75887bf5515f0b0ccde13'), ObjectId('67c75893bf5515f0b0ccde31'), ObjectId('67c75893bf5515f0b0ccde33'), ObjectId('67c75893bf5515f0b0ccde39'), ObjectId('67c75893bf5515f0b0ccde3b'), ObjectId('67d27b44cbe1d16869afdcc0'), ObjectId('67d27b44cbe1d16869afdcc2'), ObjectId('67d27b44cbe1d16869afdcc4'), ObjectId('67d27b44cbe1d16869afdcc6'), ObjectId('67d27b45cbe1d16869afdccf'), ObjectId('67d27b45cbe1d16869afdcd1'), ObjectId('6

In [27]:
print(id[0])

TypeError: 'builtin_function_or_method' object is not subscriptable

In [ ]:
from pymongo import MongoClient
from bson import ObjectId  # Import ObjectId

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["postbuddy"]
collection = db["posts"]

# Correct way to filter using ObjectId
post_id = ObjectId("67c756e2bf5515f0b0ccddb5")  # Convert string to ObjectId
document = collection.find_one({"_id": post_id})

if document:
    print(document)
else:
    print("No document found with this _id.")


In [1]:
from pymongo import MongoClient
import os 
# Connect to MongoDB
uri = os.getenv("MONGO_URI")
client = MongoClient(uri)
  # Change if needed
db = client["postbuddy"]
collection = db["posts"]
# Define the fields to be added with empty values
update_fields = {
    "category": "",
    "refinedDescription": ""
}

# Update all documents that do not have these fields
collection.update_many(
    {"category": {"$exists": False}},  # Ensures we only update documents missing this field
    {"$set": update_fields}
)

print("Documents updated successfully!")


Documents updated successfully!


In [19]:
import os
import logging
from pymongo import MongoClient, errors
from dotenv import load_dotenv

from enum import Enum
# Load environment variables (if using .env)
load_dotenv()
# Get MongoDB URI from environment variable
uri = os.getenv("MONGO_URI")

if not uri:
    logging.error("MONGO_URI is not set. Please configure it in environment variables.")
    raise ValueError("MONGO_URI is not set. Please configure it in your environment variables.")

try:
    # Connect to MongoDB
    client = MongoClient(uri, serverSelectionTimeoutMS=5000)
    client.admin.command("ping")  
    logging.info("Connected to MongoDB successfully.")

    # Select database and collection
    db = client["postbuddy"]
    collection = db["posts"]

    # Fetch all documents
    documents = collection.find()

    print(documents[1])
    # Print documents
    # Loop through documents and extract specific fields
    for doc in documents:
        post_id = doc["_id"]
        post_title = doc["postTitle"]
        description = doc["description"]
        post_url = doc["postUrl"]
        image_url = doc.get("imageUrl", "No Image")  # Use .get() to avoid KeyError
        platform = doc["platform"]
        
        processed = doc["processed"]

        # Print extracted values
        print(f"Post ID: {post_id}")
        print(f"Title: {post_title}")
        print(f"Description: {description[:100]}...")  # Limit description length
        print(f"Post URL: {post_url}")
        print(f"Image URL: {image_url}")
        print(f"Platform: {platform}")
   
        print(f"Processed: {processed}")
        print("-" * 50)  # Separator for readability


except errors.ServerSelectionTimeoutError as e:
    logging.error("Failed to connect to MongoDB: Server selection timeout.", exc_info=True)
    raise ConnectionError("Failed to connect to MongoDB.") from e
except errors.ConnectionFailure as e:
    logging.error("MongoDB connection failed.", exc_info=True)
    raise ConnectionError("MongoDB connection failed.") from e
except Exception as e:
    logging.error(f"An unexpected error occurred: {e}", exc_info=True)
    raise RuntimeError(f"An unexpected error occurred: {e}") from e


{'_id': ObjectId('67c756e2bf5515f0b0ccddb7'), 'postTitle': 'mortgage broker forgot to lock my rate', 'description': 'im currently at 7.375%. in mid-september i was in the process of refinancing for 5.625%. on my loan estimate, the rate was not locked. i expressed concerns about this, and the mortgage broker texted me saying that he locked it in after that. we were supposed to close on October 11th and he has been ignoring all my calls/texts for the past 2 weeks. i said i would file a complaint with the CFPB if i don’t hear back from him today. he called me back immediately and admitted that he forgot to lock the rate and rates have been skyrocketing since then. he said he would pay the points and lose money on this transaction to fix his mistake and get me 5.625 back. he said he would send the new loan estimate by this afternoon, but he never sent it and didn’t say anything. im not trying to be a karen but im anxious that im going to get ghosted again😭 how long should i wait?\n\nUPDATE

In [12]:
# Loop through documents and extract specific fields
for doc in documents:
    post_id = doc["_id"]
    post_title = doc["postTitle"]
    description = doc["description"]
    post_url = doc["postUrl"]
    image_url = doc.get("imageUrl", "No Image")  # Use .get() to avoid KeyError
    platform = doc["platform"]
    user_id = doc["userId"]
    last_fetched = doc["last_fetched"]
    created_at = doc["createdAt"]
    updated_at = doc["updatedAt"]
    processed = doc["processed"]

    # Print extracted values
    print(f"Post ID: {post_id}")
    print(f"Title: {post_title}")
    print(f"Description: {description[:100]}...")  # Limit description length
    print(f"Post URL: {post_url}")
    print(f"Image URL: {image_url}")
    print(f"Platform: {platform}")
    print(f"User ID: {user_id}")
    print(f"Last Fetched: {last_fetched}")
    print(f"Created At: {created_at}")
    print(f"Updated At: {updated_at}")
    print(f"Processed: {processed}")
    print("-" * 50)  # Separator for readability

In [9]:
import os
import google.generativeai as genai
import PIL.Image
from dotenv import load_dotenv  # Ensure environment variables are loaded

# Load environment variables
load_dotenv()

# Ensure API key is set
KEY = os.getenv("GEMINI_API_KEY")
if not KEY:
    raise ValueError("GEMINI_API_KEY is not set. Please check your environment variables.")

# Configure Gemini API
genai.configure(api_key=KEY)

# Load image
image = PIL.Image.open("processed_image.png")

# Initialize Gemini model
model = genai.GenerativeModel("gemini-2.0-flash")  # Correct model name

# Generate response
response = model.generate_content(["What is this image?", image])

# Print response
print(response.text)


The image is a meme. It combines a quote from the European Central Bank (ECB) claiming that early Bitcoin investors are "stealing wealth" from latecomers with a picture of Greta Thunberg and the text "HOW DARE YOU!" This suggests a sarcastic or critical response to the ECB's statement.



In [17]:
def latest_value_with_timestamp(a, b):
    if not isinstance(a, tuple) or len(a) != 2:
        a = (None, 0)  # Default value with an old timestamp
    if not isinstance(b, tuple) or len(b) != 2:
        b = (None, 0)
    
    # Ensure we only keep non-empty or meaningful values with the latest timestamp
    if b[1] > a[1]:
        if b[0] is not None and (not isinstance(b[0], (str, list)) or len(b[0]) > 0):
            return b
    return a

def get_timestamp():
    return int(time() * 1_000_000)  # Convert to microseconds


In [ ]:
from typing import Dict, List, TypedDict
from langgraph.graph import StateGraph, END, START

from dotenv import load_dotenv

from enum import Enum
# Load environment variables (if using .env)
load_dotenv()
# Define a simple GraphState TypedDict for the workflow

class GraphState(TypedDict):
    posts:Annotated[Tuple[dict[dict], int], latest_value_with_timestamp] # each post added as dictonary with fields 
    

    question: Annotated[Tuple[str, int], latest_value_with_timestamp]


    # context: Annotated[Tuple[str, int], latest_value_with_timestamp]
    # current_chunk_count: Annotated[Tuple[int, int], latest_value_with_timestamp]
    # user_prompt: Annotated[Tuple[str, int], latest_value_with_timestamp]
    # answer_generator_model_name: Annotated[Tuple[str, int], latest_value_with_timestamp]
    # answer: Annotated[Tuple[str, int], latest_value_with_timestamp]
    # user_id: Annotated[Tuple[str, int], latest_value_with_timestamp]  # Added to track requests
    # sub_prompts: Annotated[list, latest_value_with_timestamp]
    # prompt_break: Annotated[Tuple[bool, int], latest_value_with_timestamp]
    # prompt_breaking_llm: Annotated[Tuple[str, int], latest_value_with_timestamp]
    # sub_prompts_created: Annotated[Tuple[bool, int], latest_value_with_timestamp]
    # current_retries_for_subprompts: Annotated[Tuple[int, int], latest_value_with_timestamp]
    # max_retries_for_subprompting: Annotated[Tuple[int, int], latest_value_with_timestamp]
    # chunk_count_min: Annotated[Tuple[int, int], latest_value_with_timestamp]
    # chunk_count_max: Annotated[Tuple[int, int], latest_value_with_timestamp]
    # first_response_for_subprompts: Annotated[Tuple[str, int], latest_value_with_timestamp]
    # prompt_refining_llm:Annotated[Tuple[str, int], latest_value_with_timestamp]
    # prompt_refine:Annotated[Tuple[bool, int], latest_value_with_timestamp]
    # refined_prompt:Annotated[Tuple[str, int], latest_value_with_timestamp]
    # current_retries_for_refining: Annotated[Tuple[int, int], latest_value_with_timestamp]
    # max_retries_for_refining: Annotated[Tuple[int, int], latest_value_with_timestamp]
    # first_response_for_refining:Annotated[Tuple[str, int], latest_value_with_timestamp]
    # default_embedding_model:Annotated[Tuple[str, int], latest_value_with_timestamp]
    # vectorDB:Annotated[Tuple[str, int], latest_value_with_timestamp]
    # collection_name:Annotated[Tuple[str, int], latest_value_with_timestamp]
    # db_handler: Annotated[Tuple[Any, int], latest_value_with_timestamp]
    # no_change_in_query:Annotated[Tuple[bool, int], latest_value_with_timestamp]

# Dummy node functions
def analyze_text(state: GraphState) -> Dict:
    print("Breaking prompt into sub-questions...")
    return {"broken_question": ["Sub-question 1", "Sub-question 2"]}

def analyze_image(state: GraphState) -> Dict:
    print("Refining the original prompt...")


    KEY = os.getenv("GEMINI_API_KEY")
    if not KEY:
        raise ValueError("GEMINI_API_KEY is not set. Please check your environment variables.")

    # Configure Gemini API
    genai.configure(api_key=KEY)

    # Load image
    image = PIL.Image.open("sunset.jpg")

    # Initialize Gemini model
    model = genai.GenerativeModel("gemini-2.0-flash")  # Correct model name

    # Generate response
    response = model.generate_content(["What is this image?", image])

    # Print response
    print(response.text)

    
    return {"refined_question": "Refined question here"}

def get_chunks(state: GraphState) -> Dict:
    print("Retrieving relevant chunks...")
    return {"context": ["Chunk 1", "Chunk 2", "Chunk 3"]}

def load_document(state: GraphState) -> Dict:
    print("Loading document...")
    return {"document": "Document here"}

# Create the graph
workflow = StateGraph(GraphState)

# Add nodes
workflow.add_node("text_analysis", analyze_text)
workflow.add_node("image_analysis", analyze_image)
workflow.add_node("storeInDB", get_chunks)
workflow.add_node("document_loader", load_document)


# Add edges
workflow.add_edge(START, "document_loader")
workflow.add_edge("document_loader", "text_analysis")
workflow.add_edge("document_loader", "image_analysis")
workflow.add_edge("text_analysis", "storeInDB")

workflow.add_edge("image_analysis", "storeInDB")

workflow.add_edge("storeInDB", END)


# Compile the graph
graph_app = workflow.compile()


image_data = graph_app.get_graph().draw_mermaid_png()
with open("LangGraph.png", "wb") as f:
    f.write(image_data)



In [22]:
from db import QdrantDB

db_handler = QdrantDB()  # Instantiate the QdrantDB class
if not db_handler.is_running():
    print("qdrant is not running")

NameError: name 'GraphState' is not defined

In [19]:
print.info(f" the context being returned now is ")

AttributeError: 'builtin_function_or_method' object has no attribute 'info'

In [28]:
import ollama
def generate_ans(user_prompt, system_prompt, model_name):


    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
 
    response = ollama.chat(
        model=model_name, 
        messages=messages, 
        options={"temperature": 0.7, "num_ctx": 7000}
    )
    return response['message']['content']
   

In [36]:
import ollama
from groq import Groq
groq_client = Groq(
    api_key="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",
)

def generate_refined_post(post_title, description, image_analysis=None):
    system_prompt = """
    You are an AI assistant trained to refine and enhance post descriptions for better semantic understanding. 
    Your goal is to take the given post details (title, description, and image analysis) and generate a 
    well-structured, coherent, and concise unified summary. The refined summary should:

    1️⃣ Maintain the **core meaning** of the post.  
    2️⃣ Improve **clarity, readability, and structure**.  
    3️⃣ Integrate **image insights naturally** if available.  
    4️⃣ Remove **irrelevant details or redundancy**.  
    5️⃣ Ensure the summary is **self-contained and informative**.  

    Keep the tone **neutral and informative**, avoiding any speculation or unnecessary commentary.
    """

    user_prompt = f"""
    Post Title: {post_title}

    Post Description: {description}

    Image Analysis: {image_analysis if image_analysis else "No image analysis available"}

    🔹 **Task:** Generate a refined, coherent, and well-structured summary that captures the **essence of the post** 
    using the provided information. Ensure clarity, conciseness, and completeness.
    """

    response = groq_client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"{system_prompt}"
            },
            {
                "role": "user",
                "content": f"{user_prompt} "
            }
        ],
        model="llama-3.3-70b-versatile",
        max_tokens=1500
    )
    return response.choices[0].message.content
    



In [38]:

post_title = "Nawaz Ashraf"
description = "I shared my graphics proposal #samples recently and got more than 200 requests to share my web proposals too. I am sharing my 10 BEST WEB PROPOSALS that helped me secure the projects on Upwork. If you want to get my winning proposals then comment \"samples\" and I will share my proposals set with you. I know I can share it publicly too but it has to be a two way exchange so comment on the post and I can share my proposal samples with you via inbox.Start your freelancing journey now and make a personal revenue stream for yourself. If you are facing any hurdles on Upwork then share your concerns with me and I will try my best to help you guys. #Upworkproposals #proposalwriting #Upworktips\n              \n\n        \n          …see more"
image_analysis = "The image shows a list of freelance job postings, likely from a platform like Upwork or Fiverr. Each entry includes the job title, a rating with star icons (all at 5.00), the date range of the project or posting, a price, and often a snippet of client feedback.  The jobs are:\n\n1.  \"Web Developer to transfer fully developed Web Site from Scorpion platform to Wordpress\" with a fixed price of $2,000.00. The date range is Jan 23, 2024 - Mar 19, 2024.\n2.  \"Wordpress Website New Pages\" with a fixed price of $1,500.00. The date range is Nov 3, 2023 - Jan 15, 2024. Client feedback mentions the freelancer (presumably named \"Nawaz\") did a great job designing their website.\n3.  \"Zoho Sites specialist to help create new website\" with a fixed price of $2,500.00. The date range is Dec 6, 2022 - Jan 25, 2023. Client feedback says Nawaz did a fantastic job and was always available.\n4.  \"Design and implement wordpress site for educational content/videos\" with a fixed price of $3,500.00. The date range is Jul 27, 2022 - Mar 4, 2023. Client feedback praises Nawaz for being responsive, taking feedback well, and completing work efficiently.\n\nAll jobs are listed as \"Fixed price\"."

refined_summary = generate_refined_post(post_title, description, image_analysis)
print(refined_summary)


Nawaz Ashraf is sharing his expertise in creating successful web proposals on Upwork. After receiving over 200 requests, he is offering to share his 10 best web proposals that have helped him secure projects. To access these proposals, interested individuals can comment "samples" on the post, and Nawaz will share them via inbox. The proposals have been instrumental in his success as a freelancer, with a track record of delivering high-quality work and receiving positive client feedback. As evident from his freelance job postings, Nawaz has worked on various projects, including web development, WordPress website design, and Zoho Sites creation, with fixed prices ranging from $1,500 to $3,500. He is also willing to help others who are facing challenges on Upwork, offering guidance and support to those starting their freelancing journey. By sharing his proposals and expertise, Nawaz aims to help others create a personal revenue stream and succeed as freelancers.


claude prompts 

In [ ]:
system_prompt="""You are a specialized post summarization assistant. Your task is to refine post descriptions to make them optimally retrievable through natural language queries. Focus on extracting and highlighting key information, entities, topics, and emotions present in the post. Format the refined description in a clear, searchable manner while preserving all important details."""Langraph.png

user_prompt=""" Below is a unified description of a saved post containing the post title, description, and image content (if present). Refine this description to optimize it for retrieval when users search using natural language queries.

Original Unified Description:
[UNIFIED_DESCRIPTION]

Please create a refined version that:
1. Highlights key topics, entities, and concepts
2. Preserves important details and context
3. Structures information in a way that's optimized for semantic search
4. Maintains the post's original intent and meaning"""

In [45]:
import ollama
from groq import Groq
groq_client = Groq(
    api_key="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",
)

def generate_refined_post(post_title, description, image_analysis=None):
    
    system_prompt="""You are a specialized post summarization assistant. Your task is to refine post descriptions to make them optimally retrievable through natural language queries. Focus on extracting and highlighting key information, entities, topics, and emotions present in the post. Format the refined description in a clear, searchable manner while preserving all important details."""

    if not image_analysis or image_analysis.strip() == "":
        image_analysis_text = "No image is present in this post."
    else:
        image_analysis_text = image_analysis

    UNIFIED_DESCRIPTION=f"""
    Post Title: {post_title}

    Post Description: {description}

    Image Analysis: {image_analysis_text}

    """

    

    user_prompt=f""" Below is a unified description of a saved post containing the post title, description, and image content (if present). Refine this description to optimize it for retrieval when users search using natural language queries.

        Original Unified Description:
        {UNIFIED_DESCRIPTION}

        Please create a refined version that:
        1. Highlights key topics, entities, and concepts
        2. Preserves important details and context
        3. Structures information in a way that's optimized for semantic search
        4. Maintains the post's original intent and meaning"""

    response = groq_client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"{system_prompt}"
            },
            {
                "role": "user",
                "content": f"{user_prompt} "
            }
        ],
        model="llama-3.3-70b-versatile",
        max_tokens=1500
    )
    return response.choices[0].message.content
    



In [46]:

import json

post_title = "Nawaz Ashraf"
description = "I shared my graphics proposal #samples recently and got more than 200 requests to share my web proposals too. I am sharing my 10 BEST WEB PROPOSALS that helped me secure the projects on Upwork. If you want to get my winning proposals then comment \"samples\" and I will share my proposals set with you. I know I can share it publicly too but it has to be a two way exchange so comment on the post and I can share my proposal samples with you via inbox.Start your freelancing journey now and make a personal revenue stream for yourself. If you are facing any hurdles on Upwork then share your concerns with me and I will try my best to help you guys. #Upworkproposals #proposalwriting #Upworktips\n              \n\n        \n          …see more"
image_analysis = "The image shows a list of freelance job postings, likely from a platform like Upwork or Fiverr. Each entry includes the job title, a rating with star icons (all at 5.00), the date range of the project or posting, a price, and often a snippet of client feedback.  The jobs are:\n\n1.  \"Web Developer to transfer fully developed Web Site from Scorpion platform to Wordpress\" with a fixed price of $2,000.00. The date range is Jan 23, 2024 - Mar 19, 2024.\n2.  \"Wordpress Website New Pages\" with a fixed price of $1,500.00. The date range is Nov 3, 2023 - Jan 15, 2024. Client feedback mentions the freelancer (presumably named \"Nawaz\") did a great job designing their website.\n3.  \"Zoho Sites specialist to help create new website\" with a fixed price of $2,500.00. The date range is Dec 6, 2022 - Jan 25, 2023. Client feedback says Nawaz did a fantastic job and was always available.\n4.  \"Design and implement wordpress site for educational content/videos\" with a fixed price of $3,500.00. The date range is Jul 27, 2022 - Mar 4, 2023. Client feedback praises Nawaz for being responsive, taking feedback well, and completing work efficiently.\n\nAll jobs are listed as \"Fixed price\"."

refined_summary = generate_refined_post(post_title, description, image_analysis)
print(refined_summary)


UNIFIED_DESCRIPTION=f"""
    Post Title: {post_title}

    Post Description: {description}

    Image Analysis: {image_analysis}

    """


DESCRIPTION = {
    "UNIFIED_DESCRIPTION": UNIFIED_DESCRIPTION,
    "LLM UNIFIED DESCRIPTION": refined_summary
}


json_filename = "unified_description.json"

# Write to JSON file
with open(json_filename, "w", encoding="utf-8") as json_file:
    json.dump(DESCRIPTION, json_file, indent=4, ensure_ascii=False)



**Refined Post Description:**

**Topic:** Freelancing, Upwork Proposals, Proposal Writing, Web Development, Freelance Jobs

**Entity:** Nawaz Ashraf, Upwork, Fiverr

**Description:** 
Nawaz Ashraf is sharing his expertise in creating winning web proposals on Upwork. After receiving over 200 requests, he is offering to share his 10 best web proposals that helped him secure projects on the platform. To access these proposal samples, users can comment "samples" on the post, and Nawaz will share them via inbox. This is a unique opportunity for freelancers to learn from his experiences and improve their own proposal writing skills.

**Key Highlights:**

* Nawaz Ashraf's web proposals have helped him secure multiple projects on Upwork
* He is sharing his 10 best web proposals with those who comment "samples" on the post
* The proposals have resulted in successful projects with high client satisfaction rates
* The jobs listed include web development, WordPress website creation, and Zoho Sites

In [43]:
import re
def remove_formatting(text: str) -> str:
    """Removes markdown-style bold and other unnecessary formatting."""
    return re.sub(r"\*\*(.*?)\*\*", r"\1", text)  # Removes **bold** formatting


clean=remove_formatting(refined_summary)
print(clean)

Refined Description:

Title: Nawaz Ashraf's Winning Web Proposals on Upwork

Description: Freelancer Nawaz Ashraf is sharing their 10 best web proposals that helped secure projects on Upwork. With over 200 requests for web proposal samples, Nawaz is offering to share their winning proposals with those who comment "samples" on the post. The proposals can be shared privately via inbox, allowing for a two-way exchange. This resource is particularly useful for freelancers looking to start their journey on Upwork and create a personal revenue stream. Nawaz is also open to helping others overcome hurdles on the platform by sharing concerns and receiving guidance.

Key Topics:

* Freelancing: Starting a freelancing journey and creating a personal revenue stream
* Upwork: Navigating the Upwork platform, overcoming hurdles, and securing projects
* Proposal Writing: Crafting winning web proposals to secure freelance work
* Web Development: Examples of web development projects, including WordPres

In [6]:
from qdrant_client import QdrantClient, models
from qdrant_client.models import VectorParams, Distance

client=QdrantClient(host="localhost", port=6333)

f:\Mini-Conda\envs\fyp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from pymongo import MongoClient
from bson import ObjectId  # Import ObjectId
ids=[ObjectId('67c756e2bf5515f0b0ccddb5'), ObjectId('67c756e2bf5515f0b0ccddb7'), ObjectId('67c756e2bf5515f0b0ccddb9'), ObjectId('67c756e2bf5515f0b0ccddbb'), ObjectId('67c756e2bf5515f0b0ccddbd'), ObjectId('67c756e3bf5515f0b0ccddbf'), ObjectId('67c75885bf5515f0b0ccddf1'), ObjectId('67c75885bf5515f0b0ccddf3'), ObjectId('67c75885bf5515f0b0ccddf5'), ObjectId('67c75885bf5515f0b0ccddf7'), ObjectId('67c75886bf5515f0b0ccddf9'), ObjectId('67c75886bf5515f0b0ccddfb'), ObjectId('67c75887bf5515f0b0ccde0a'), ObjectId('67c75887bf5515f0b0ccde0c'), ObjectId('67c75887bf5515f0b0ccde0e'), ObjectId('67c75887bf5515f0b0ccde13'), ObjectId('67c75893bf5515f0b0ccde31'), ObjectId('67c75893bf5515f0b0ccde33'), ObjectId('67c75893bf5515f0b0ccde39'), ObjectId('67c75893bf5515f0b0ccde3b'), ObjectId('67d27b44cbe1d16869afdcc0'), ObjectId('67d27b44cbe1d16869afdcc2'), ObjectId('67d27b44cbe1d16869afdcc4'), ObjectId('67d27b44cbe1d16869afdcc6'), ObjectId('67d27b45cbe1d16869afdccf'), ObjectId('67d27b45cbe1d16869afdcd1'), ObjectId('67d27d6ecbe1d16869afdcef'), ObjectId('67d27d6ecbe1d16869afdcf1'), ObjectId('67d27d6fcbe1d16869afdcf7'), ObjectId('67d27d6fcbe1d16869afdcf9'), ObjectId('67d27d6ecbe1d16869afdced'), ObjectId('67d27d70cbe1d16869afdcfd')]

In [ ]:
post_ids = [str(id) for id in ids]
post_ids


In [36]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
embed_model = FastEmbedEmbeddings()

# single_text="Retrive me the post on timetable solution"
single_text="""Title: Building a SaaS Business with Flux AI in 3 Days as a 17-Year-Old

Description: 
A 17-year-old entrepreneur, Sebastian, created a consumer interface for the AI image generator Flux, called *TheFluxAI*, after discovering the API had no user-friendly interface and required payment for use. 
"""
query_vector= embed_model.embed_query(single_text)

In [37]:
post_ids = [str(pid) for pid in post_ids]

        # Perform vector search filtered by the allowed post_ids
results_from_db = client.search(
    collection_name="my_collection",
    query_vector=query_vector,
    with_payload=True,
    limit=30,
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.post_id",  # Direct key, not nested under "metadata"
                match=models.MatchAny(any=post_ids)
            )
        ]
    )
)

matched_post_ids=[]
for point in results_from_db:
    matched_post_ids.append(point.payload["metadata"]["post_id"])

for id in matched_post_ids:
    print(id)


C:\Users\MY IT STORE\AppData\Local\Temp\ipykernel_9280\1213588080.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results_from_db = client.search(


67c75885bf5515f0b0ccddf7
67c75893bf5515f0b0ccde3b
67d27b44cbe1d16869afdcc4
67d27b45cbe1d16869afdccf
67c756e2bf5515f0b0ccddbd
67c75893bf5515f0b0ccde31
67d27d70cbe1d16869afdcfd
67c75887bf5515f0b0ccde0c
67c756e2bf5515f0b0ccddbd
67c756e2bf5515f0b0ccddbd
67c756e2bf5515f0b0ccddbd
67c756e2bf5515f0b0ccddbd
67c756e2bf5515f0b0ccddbd
67c756e2bf5515f0b0ccddbd
67c756e3bf5515f0b0ccddbf
67c756e2bf5515f0b0ccddbd
67c756e2bf5515f0b0ccddbd
67c75893bf5515f0b0ccde33
67c756e2bf5515f0b0ccddbd
67c75887bf5515f0b0ccde13
67c75887bf5515f0b0ccde0a
67c756e3bf5515f0b0ccddbf
67c756e2bf5515f0b0ccddbd
67d27d6ecbe1d16869afdcef
67c756e2bf5515f0b0ccddbd
67c75887bf5515f0b0ccde0e
67d27b44cbe1d16869afdcc2
67c756e2bf5515f0b0ccddb7
67c75886bf5515f0b0ccddf9
67c75885bf5515f0b0ccddf1


In [ ]:



print(len(matched_post_ids))

30


In [15]:
print(results_from_db)
print(len(results_from_db))

[ScoredPoint(id='117f5296-fbd8-41e4-a814-628f1401c1ee', version=12, score=0.6201962, payload={'text': "Post Title: Daniyal Imran Joins CodesOrbit as MERN Stack Developer\nPost Description: Daniyal Imran is excited to announce his new role as a MERN Stack Developer at CodesOrbit. This significant step in his career journey is a result of the support and guidance he received from Amal Tariq during the hiring process. Daniyal is looking forward to contributing to innovative projects and growing with the talented team at CodesOrbit. He expresses his gratitude for the opportunity and is enthusiastic about this new beginning in his career.\n\nKey Topics: Career Development, MERN Stack, CodesOrbit, New Job, Technology\nEntities: Daniyal Imran, Amal Tariq, CodesOrbit\nConcepts: Career Growth, Innovation, Teamwork, Gratitude\nEmotions: Excitement, Gratitude, Enthusiasm\n\nImage Analysis: The accompanying image is a celebratory illustration featuring balloons and a smiling sun, set against a lig

In [38]:
id="67dbd9706101f8917561afe3"
for ids in matched_post_ids:
    if ids==id:
        print("id found")
        

In [26]:
c=0
for id in matched_post_ids:
    for fid in post_ids:
        if id==fid:
            c+=1

c

30

In [27]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
embed_model = FastEmbedEmbeddings()

# single_text="Retrive me the post on timetable solution"
single_text="Daniyal Imran Joins CodesOrbit as MERN Stack Developer\nPost D"
query_vector= embed_model.embed_query(single_text)

In [10]:
# Vector Search (Qdrant)
from qdrant_client import QdrantClient, models
from qdrant_client.models import VectorParams, Distance

client=QdrantClient(host="localhost", port=6333)
results_from_db=client.query_points(
            collection_name="my_collection",
            query=query_vector,
            with_payload=True,
            limit=10,
            # query_filter=models.Filter(
            #     must=[
            #         models.FieldCondition(
            #             key="user_id",
            #             match=models.MatchValue(value="user_00"),
            #         )
            #     ]
            # )
            )

In [11]:
for point in results_from_db.points:
    print(point.payload)

{'text': "Post Title: Daniyal Imran Joins CodesOrbit as MERN Stack Developer\nPost Description: Daniyal Imran is excited to announce his new role as a MERN Stack Developer at CodesOrbit. This significant step in his career journey is a result of the support and guidance he received from Amal Tariq during the hiring process. Daniyal is looking forward to contributing to innovative projects and growing with the talented team at CodesOrbit. He expresses his gratitude for the opportunity and is enthusiastic about this new beginning in his career.\n\nKey Topics: Career Development, MERN Stack, CodesOrbit, New Job, Technology\nEntities: Daniyal Imran, Amal Tariq, CodesOrbit\nConcepts: Career Growth, Innovation, Teamwork, Gratitude\nEmotions: Excitement, Gratitude, Enthusiasm\n\nImage Analysis: The accompanying image is a celebratory illustration featuring balloons and a smiling sun, set against a light blue background with scattered white dots, resembling confetti or snowflakes. The image co

## testing endpoint request

In [1]:
import requests

url = "https://abd6-202-166-164-217.ngrok-free.app/chat-with-post"
payload = {
    "user_id": "67c75778bf5515f0b0ccddc2",
    "post_id": "67c756e2bf5515f0b0ccddbb",
    "user_query": "What is this post about?"
}

response = requests.post(url, json=payload)

print("Status:", response.status_code)

try:
    print("JSON Response:", response.json())
except requests.exceptions.JSONDecodeError:
    print("Non-JSON Response:", response.text)


Status: 200
JSON Response: {'status': 'success', 'chat_id': '67c75778bf5515f0b0ccddc2:67c756e2bf5515f0b0ccddbb', 'system_response': "This post is about a proposal for Microsoft to invest in Bitcoin that is being put to a vote by the company's shareholders. The key points are:\n\n* The proposal is to be voted on at the Microsoft shareholder meeting on December 10.\n* The Microsoft Board of Directors has recommended that shareholders vote against the proposal.\n* The vote will determine whether Microsoft invests in Bitcoin, which could have implications for the company's investment strategy and involvement in the cryptocurrency market.\n\nOverall, the post is about the potential for Microsoft to invest in Bitcoin and the upcoming shareholder vote that will decide the outcome."}
